In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import pickle as pickle
import pandas as pd
from scipy.stats import skew,kurtosis
import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,RepeatedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.utils import shuffle

import os
import tensorflow as tf
import time
import sys
sys.path.append("..")
# import autosklearn.classification

In [ ]:
A = np.load('/content/drive/MyDrive/Thesis data/A.npy') # File with all(40) channel data
B = np.load('/content/drive/MyDrive/Thesis data/B.npy') #File with only data from left hemisphere 
A_norm = np.load('/content/drive/MyDrive/Thesis data/Anorm.npy') # normalised 40 channel data
B_norm = np.load('/content/drive/MyDrive/Thesis data/Anorm.npy') # normalised 7 channel data
print("A.shape =",A.shape)
print("B.shape =",B.shape)
#Load the labels now    
valence = np.genfromtxt('/content/drive/MyDrive/Thesis data/label_class_0.dat', delimiter=' ')
arousal = np.genfromtxt('/content/drive/MyDrive/Thesis data/label_class_1.dat', delimiter=' ')
dominance = np.genfromtxt('/content/drive/MyDrive/Thesis data/label_class_2.dat', delimiter=' ')
liking = np.genfromtxt('/content/drive/MyDrive/Thesis data/label_class_3.dat', delimiter=' ')


A.shape = (51200, 99)
B.shape = (8960, 99)


## Augmenting data below
Start with the labels<br>
For example va2 means valence augmented twice<br>
ar3 means arousal augmented thrice

In [ ]:
###################################
# Augmenting the labels 
# 2x
va2 = np.hstack((valence,valence))
ar2 = np.hstack((arousal,arousal))
do2 = np.hstack((dominance,dominance))
li2 = np.hstack((liking,liking))


# 3x
va3 = np.hstack((valence,valence,valence))
ar3 = np.hstack((arousal,arousal,arousal))
do3 = np.hstack((dominance,dominance,dominance))
li3 = np.hstack((liking,liking,liking))

# 4x
va4 = np.hstack((valence,valence,valence,valence))
ar4 = np.hstack((arousal,arousal,arousal,arousal))
do4 = np.hstack((dominance,dominance,dominance,dominance))
li5 = np.hstack((liking,liking,liking,liking))


# 5x
va5 = np.hstack((valence,valence,valence,valence,valence))
ar5 = np.hstack((arousal,arousal,arousal,arousal,arousal))
do5 = np.hstack((dominance,dominance,dominance,dominance,dominance))
li5 = np.hstack((liking,liking,liking,liking,liking))

# 10x
va10 = np.hstack((valence,valence,valence,valence,valence,valence,valence,valence,valence,valence))
ar10 = np.hstack((arousal,arousal,arousal,arousal,arousal,arousal,arousal,arousal,arousal,arousal))
do10 = np.hstack((dominance,dominance,dominance,dominance,dominance,dominance,dominance,dominance,dominance,dominance))
li10 = np.hstack((liking,liking,liking,liking,liking,liking,liking,liking,liking,liking))

## Get some generated data points<br>
Get the data points with generator and discriminator close to zero and least as possible

In [ ]:
g1 =np.load('/content/drive/MyDrive/Thesis data/Generated data/Generated_ep78.npy') # generated data with all(40) channel data

g2 =np.load('/content/drive/MyDrive/Thesis data/Generated data/Generated_ep80.npy') # generated data with all(40) channel data

g3 =np.load('/content/drive/MyDrive/Thesis data/Generated data/Generated_ep86.npy') # generated data with all(40) channel data

g4 =np.load('/content/drive/MyDrive/Thesis data/Generated data/Generated_ep88.npy') # generated data with all(40) channel data

g5 =np.load('/content/drive/MyDrive/Thesis data/Generated data/Generated_ep90.npy') # generated data with all(40) channel data

g6 =np.load('/content/drive/MyDrive/Thesis data/Generated data/Generated_ep92.npy') # generated data with all(40) channel data

g7 =np.load('/content/drive/MyDrive/Thesis data/Generated data/Generated_ep93.npy') # generated data with all(40) channel data

g8 =np.load('/content/drive/MyDrive/Thesis data/Generated data/Generated_ep94.npy') # generated data with all(40) channel data

g9 =np.load('/content/drive/MyDrive/Thesis data/Generated data/Generated_ep95.npy') # generated data with all(40) channel data

g10 =np.load('/content/drive/MyDrive/Thesis data/Generated data/Generated_ep100.npy') # generated data with all(40) channel data


g1.shape

(1280, 40, 99)

## Augment the data

In [ ]:
###################################
# Augmenting the data 
print("Augmenting data............")

# 2x
data2 = np.concatenate([g7, A_norm], 0) 
print(data2.shape)

# 3x
data3 = np.concatenate([g7,g1, A_norm], 0)
print(data3.shape)

# 4x
data4 = np.concatenate([g7,g1,g2, A_norm], 0)
print(data4.shape)


# 5x
data5 = np.concatenate([g7,g1,g2,g3, A_norm], 0)
print(data5.shape)

# 10x
data10 = np.concatenate([g1,g2,g3,g4,g5,g6,g7,g8,g9, A_norm], 0)
print(data10.shape)

Augmenting data............
(2560, 40, 99)
(3840, 40, 99)
(5120, 40, 99)
(6400, 40, 99)
(12800, 40, 99)


In [ ]:
# Some metrics to be displayed
conf_matrix = 1
cls_report = 1
auc_report = 1

In [ ]:
# X = input_data = A_norm.reshape(32*40,40*99) 
# y = valence

# cv_inner = KFold(n_splits=4, shuffle=True, random_state=1)
# # define the model
# model = KNeighborsClassifier(n_jobs=-1)
# #Hyper Parameters Set
# params = {'n_neighbors':[1,5,6,7,8,9,10,20],
#         'leaf_size':[1,2,3,5],
#         'weights':['uniform', 'distance'],
#         'algorithm':['auto', 'ball_tree','kd_tree','brute'],
#         'n_jobs':[-1]}
# # define search
# model1 = GridSearchCV(model, params, scoring='accuracy', n_jobs=1, cv=cv_inner, refit=True)
# # configure the cross-validation procedure
# cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
# # execute the nested cross-validation
# scores = cross_val_score(model1, X, y, scoring='accuracy', cv=cv_outer, n_jobs=-1)
# # report performance
# print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

## K fold based 

### Actual data

In [ ]:
X = input_data = A_norm.reshape(32*40,40*99) 
conf_matrix = 0
cls_report = 0

In [ ]:
# This is the main K fold block 
y = valence
# Split the data into training/testing sets
acc_per_fold = []
fold_no = 1
kval = 20
dispresults = 0
nsplit = 32
shuffle = False
dispfoldres = 0
print()
print("################# Valence #################")
kfold = KFold(n_splits=nsplit,shuffle=shuffle) # 4,8,32
for train, test in kfold.split(X, y):
  # print(X[train].shape, X[test].shape)
  X_train = X[train]
  y_train = y[train]
  X_test = X[test]
  y_test = y[test]


  # KNN
  clf = KNeighborsClassifier(n_neighbors=kval) #RandomForestClassifier(n_jobs=-1,random_state=123)# DecisionTreeClassifier(random_state=123) # clf = KNeighborsClassifier(n_neighbors=1) 
  clf.fit(X_train, y_train)
  y_predict = clf.predict(X_test)
  acc = accuracy_score(y_test, y_predict)*100
  acc = round(acc, 4)
  if dispresults:
    print("Accuracy score for fold",fold_no)
    print(acc)
    print('\n')
  acc_per_fold.append(acc)
  if conf_matrix:
      print(confusion_matrix(y_test, y_predict))
  if cls_report:
      print(classification_report(y_test, y_predict))
  
  fold_no = fold_no + 1
if dispfoldres:  
  print('------------------------------------------------------------------------')
  print('Score per fold')
  for i in range(0, len(acc_per_fold)):
      print('------------------------------------------------------------------------')
      print(f'> Fold {i+1} Accuracy: {acc_per_fold[i]}%')
  print('------------------------------------------------------------------------')
print("For K value -",kval, "and nsplits",nsplit,"shuffle -",shuffle)
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print('------------------------------------------------------------------------')


  # ###############################################################################
y = arousal
# Split the data into training/testing sets
acc_per_fold = []
fold_no = 1
print("################# Arousal #################")
kfold = KFold(n_splits=nsplit,shuffle=shuffle) # 4,8,32
for train, test in kfold.split(X, y):
  # print(X[train].shape, X[test].shape)
  X_train = X[train]
  y_train = y[train]
  X_test = X[test]
  y_test = y[test]
  
    
  # KNN
  clf = KNeighborsClassifier(n_neighbors=kval) #RandomForestClassifier(n_jobs=-1,random_state=123)# DecisionTreeClassifier(random_state=123) # KNeighborsClassifier(n_neighbors=1) 
  clf.fit(X_train, y_train)
  y_predict = clf.predict(X_test)
  acc = accuracy_score(y_test, y_predict)*100
  acc = round(acc, 4)
  if dispresults:
    print("Accuracy score for fold",fold_no)
    print(acc)
    print('\n')
  acc_per_fold.append(acc)
  if conf_matrix:
      print(confusion_matrix(y_test, y_predict))
  if cls_report:
      print(classification_report(y_test, y_predict))
  
  fold_no = fold_no + 1

if dispfoldres:  
  print('------------------------------------------------------------------------')
  print('Score per fold')
  for i in range(0, len(acc_per_fold)):
      print('------------------------------------------------------------------------')
      print(f'> Fold {i+1} Accuracy: {acc_per_fold[i]}%')
  print('------------------------------------------------------------------------')
print("For K value -",kval, "and nsplits",nsplit,"shuffle -",shuffle)
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print('------------------------------------------------------------------------')

  # ###############################################################################
y = dominance
# Split the data into training/testing sets
acc_per_fold = []
fold_no = 1
print("################# Dominance #################")
kfold = KFold(n_splits=nsplit,shuffle=shuffle) # 4,8,32
for train, test in kfold.split(X, y):
  # print(X[train].shape, X[test].shape)
  X_train = X[train]
  y_train = y[train]
  X_test = X[test]
  y_test = y[test]

    
  # KNN
  clf = KNeighborsClassifier(n_neighbors=kval) #RandomForestClassifier(n_jobs=-1,random_state=123)# DecisionTreeClassifier(random_state=123) # clf = KNeighborsClassifier(n_neighbors=1) 
  clf.fit(X_train, y_train)
  y_predict = clf.predict(X_test)
  acc = accuracy_score(y_test, y_predict)*100
  acc = round(acc, 4)
  if dispresults:
    print("Accuracy score for fold",fold_no)
    print(acc)
    print('\n')
  acc_per_fold.append(acc)
  if conf_matrix:
      print(confusion_matrix(y_test, y_predict))
  if cls_report:
      print(classification_report(y_test, y_predict))
  
  fold_no = fold_no + 1

if dispfoldres:  
  print('------------------------------------------------------------------------')
  print('Score per fold')
  for i in range(0, len(acc_per_fold)):
      print('------------------------------------------------------------------------')
      print(f'> Fold {i+1} Accuracy: {acc_per_fold[i]}%')
  print('------------------------------------------------------------------------')
print("For K value -",kval, "and nsplits",nsplit,"shuffle -",shuffle)
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print('------------------------------------------------------------------------')

  # ###############################################################################

y = liking
# Split the data into training/testing sets
acc_per_fold = []
fold_no = 1
print("#################Liking#################")
kfold = KFold(n_splits=nsplit,shuffle=shuffle) # 4,8,32
for train, test in kfold.split(X, y):
  # print(X[train].shape, X[test].shape)
  X_train = X[train]
  y_train = y[train]
  X_test = X[test]
  y_test = y[test]

    
  # KNN
  clf = KNeighborsClassifier(n_neighbors=kval) #RandomForestClassifier(n_jobs=-1,random_state=123)# DecisionTreeClassifier(random_state=123) # clf = KNeighborsClassifier(n_neighbors=1) 
  clf.fit(X_train, y_train)
  y_predict = clf.predict(X_test)
  acc = accuracy_score(y_test, y_predict)*100
  acc = round(acc, 4)
  if dispresults:
    print("Accuracy score for fold",fold_no)
    print(acc)
    print('\n')
  acc_per_fold.append(acc)
  
  if conf_matrix:
      print(confusion_matrix(y_test, y_predict))
  if cls_report:
      print(classification_report(y_test, y_predict))
  
  fold_no = fold_no + 1
if dispfoldres:  
  print('------------------------------------------------------------------------')
  print('Score per fold')
  for i in range(0, len(acc_per_fold)):
      print('------------------------------------------------------------------------')
      print(f'> Fold {i+1} Accuracy: {acc_per_fold[i]}%')
  print('------------------------------------------------------------------------')
print("For K value -",kval, "and nsplits",nsplit,"shuffle -",shuffle)
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print('------------------------------------------------------------------------')




################# Valence #################
For K value - 20 and nsplits 32 shuffle - False
Average scores for all folds:
> Accuracy: 52.890625 (+- 7.858846900746635)
------------------------------------------------------------------------
################# Arousal #################
For K value - 20 and nsplits 32 shuffle - False
Average scores for all folds:
> Accuracy: 51.40625 (+- 12.746591149695671)
------------------------------------------------------------------------
################# Dominance #################
For K value - 20 and nsplits 32 shuffle - False
Average scores for all folds:
> Accuracy: 55.9375 (+- 10.28405166994021)
------------------------------------------------------------------------
#################Liking#################
For K value - 20 and nsplits 32 shuffle - False
Average scores for all folds:
> Accuracy: 62.265625 (+- 10.238548767250904)
------------------------------------------------------------------------


Running different trials, best results obtained so far was <br>
55.156%  57.031%  59.609%  66.796% for valence arousal dominance and liking labels


Similar experiments were done for generated data and also the data from only left hemisphere. Results are shown below.

### Generated data

In [3]:
# Results for generated data 
# Accuracy of  Valence 52.37723281607032 (+- 8.160543056012836)
# Accuracy of  Arousal 52.82366136088967 (+- 12.77826730998265)
# Accuracy of  Dominance 50.63616163097322 (+- 11.841633770384124)
# Accuracy of  Liking 56.752233393490314 (+- 14.056293990827761)


### 7 channel data from left hemisphere

In [1]:
# Accuracy of  Valence 52.37723281607032 (+- 8.160543056012836)
# Accuracy of  Arousal 52.82366136088967 (+- 12.77826730998265)
# Accuracy of  Dominance 50.63616163097322 (+- 11.841633770384124)
# Accuracy of  Liking 56.752233393490314 (+- 14.056293990827761)
